In [20]:
import numpy as np
import json

In [60]:
with open("dataset_infos.json", "r") as json_file:
    datasets = json.load(json_file)

In [58]:
#data_list = datasets
## Create a new dictionary with the desired structure
#dataset_dict = {}
#for data_dict in data_list:
#    if 'dataset' in data_dict:
#        dataset_name = data_dict['dataset']
#        del data_dict['dataset']
#        dataset_dict[dataset_name] = data_dict

In [59]:
#with open("dataset_infos.json", "w") as json_file:
#    json.dump(dataset_dict, json_file)

In [74]:
def generate_dictionaries_with_dependency(input_dict, dependency_rule):
    import itertools

    # Separate the keys and values, and prepare for combinations excluding the dependent key
    keys, values = zip(*((k, v) for k, v in input_dict.items() if k != "latent_dim"))
    combinations = list(itertools.product(*values))

    # Create dictionaries with combinations of values and add the dependent key-value pair
    result_dicts = []
    for combination in combinations:
        new_dict = dict(zip(keys, combination))
        # Apply the dependency rule to determine the value of the dependent key
        new_dict["latent_dim"] = dependency_rule(new_dict["dataset"])
        result_dicts.append(new_dict)

    return result_dicts

# Define a rule for the dependency of 'key4' on the value of 'key1'
def dependency_rule(value_of_key1: str):
    dataset = value_of_key1.split('_')[1]
    n_features = datasets[dataset]['n_features']
    if  n_features > 128:
        latent_dim = 64
    else:
        latent_dim = n_features/2
    return int(latent_dim)

In [89]:
predefined_lists_with_dependency = {
    "dataset": ["01_ALOI", "02_annthyroid", "03_backdoor"],
    "kernel": ["rbf", "matern"],
    "batch_size": ["128"],
    "learning_rate": ["0.01", "0.001"],
    "latent_dim": [],
    "loss": ["normal", "loe_hard", "loe_soft"],
    "activation": ["relu", "tanh"],
    "labeled_anomalies": ["0.05", "0.1", "0.3", "0.9", "1.0"],
    "noise_type": ["global", "local"],
    "layers": ["5,5", "10,10" , "5,5,5"],
    "n_epochs": ["5000"]  
}
generated_dictionaries_with_dependency = generate_dictionaries_with_dependency(predefined_lists_with_dependency, dependency_rule)

In [90]:
len(generated_dictionaries_with_dependency)

2160

In [92]:
with open("experiments.json", "w") as json_file:
    json.dump(generated_dictionaries_with_dependency, json_file)

In [9]:
import pandas as pd

df = pd.read_csv(
    "/home/jovyan/work/OE-GPLVM/dataset_mapping.csv",
    names=["dataset", "n_samples", "n_features", "n_anomaly", "pct_anomaly", "domain"],
)

In [12]:
import json
datasets = json.loads(df.to_json(orient="records"))

In [15]:
with open(file_name, "w") as json_file:
    json.dump(data, json_file)

In [16]:
file_name = "dataset_infos.json"

# Use json.dump() with the 'indent' parameter set to 4
with open(file_name, "w") as json_file:
    json.dump(datasets, json_file, indent=4)